# IMDb（Internet Movie Database）からDataLoaderを作成
感情分析（０：ネガティヴ、１：ポジティブ）を２地クラス分類するためのDatasetとDataLoaderを作成する

## 目標
テキスト形式のファイルデータからtsvファイルを作成し、torchtext用のDataLoaderを作成できるようになる

## IMDbデータセットをtsv形式に変換

In [11]:
# ダウンロードしたtxtファイルをtsv形式のファイルにする

import glob
import os
import io
import string

# 訓練データのtsvファイルを作成
f = open('./data/IMDb_train.tsv', 'w')  # 書き込んでいくので'w'モード

path = './data/aclImdb/train/pos/'
for fname in glob.glob(os.path.join(path, '*.txt')):
    with io.open(fname, 'r', encoding='utf-8') as ff:
        text = ff.readline()
        # タブがあれば消す
        text = text.replace('\t', ' ')
        text = text+'\t'+'1'+'\t'+'\n'  # postive : 1
        f.write(text)  # IMDb_train.tsvに書き込む
        
path = './data/aclImdb/train/neg/'
for fname in glob.glob(os.path.join(path, '*.txt')):
    with io.open(fname, 'r', encoding='utf-8') as ff:
        text = ff.readline()
        # タブがあれば消す
        text = text.replace('\t', ' ')
        text = text+'\t'+'0'+'\t'+'\n'  # negative : 0
        f.write(text)  # IMDb_train.tsvに書き込む

f.close()

In [12]:
print(len(glob.glob(os.path.join(path, '*.txt'))))

12500


In [13]:
# テストデータのtsvフィアルを作成
f = open('./data/IMDb_test.tsv', 'w')  # 書き込んでいくので'w'モード

path = './data/acllmdb/test/pos'
for fname in glob.glob(os.path.join(path, '*.txt')):
    with io.open(fname, 'r', encoding='utf-8') as ff:
        text = ff.readline()
        # タブがあれば消す
        text = text.replace('\t', ' ')
        text = text+'\t'+'1'+'\t'+'\n'  # postive : 1
        f.write(text)  # IMDb_train.tsvに書き込む
        
path = './data/acllmdb/test/neg'
for fname in glob.glob(os.path.join(path, '*.txt')):
    with io.open(fname, 'r', encoding='utf-8') as ff:
        text = ff.readline()
        # タブがあれば消す
        text = text.replace('\t', ' ')
        text = text+'\t'+'0'+'\t'+'\n'  # negative : 0
        f.write(text)  # IMDb_train.tsvに書き込む

f.close()

## 前処理と単語分割の関数を定義
datasetを作る際のFieldごとの処理を定義するtorchtext.data.Fieldの引数にこの関数が必要

In [14]:
import string
import re

# 以下の記号をスペースに置き換える
print('区切り文字：', string.punctuation)

区切り文字： !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [15]:
# 前処理
def preprocessing_text(text):
    # 改行コードを消去
    text = re.sub('<br />', '', text)
    
    # 区切り文字の中でカンマ、ピリオド以外の記号をスペースに変換
    for p in string.punctuation:
        if (p=='.') or (p==','):
            continue
        else:
            text = text.replace(p, ' ')
            
        # ピリオドなどの前後にはすぺーすをいれておく　
        text = text.replace('.', ' . ')
        text = text.replace(',', ' , ')
        return text
    
    
# 分かち書き
# 日本語だったらmecab, janomeを使うが
# データが英語なので簡易的にスペースで区切る
def tokenizer_punctuation(text):
    return text.strip().split()


# 前処理と分かち書き
def tokenizer_with_preprocessing(text):
    text = preprocessing_text(text)
    ret = tokenizer_punctuation(text)
    return ret

# 動作確認
print(tokenizer_with_preprocessing('I like pokemon.'))

['I', 'like', 'pokemon', '.']


## DataLoaderの作成

In [16]:
# Fieldの作成
import torchtext

max_length = 256
# init_token="<cls>", eos_token="<eos>"　　文頭に<cls>, 文末に<eos>を追加しなさいという命令
TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer_with_preprocessing, use_vocab=True,
                            lower=True, include_lengths=True, batch_first=True, fix_length=max_length, init_token="<cls>", eos_token="<eos>")
LABEL = torchtext.data.Field(sequential=False, use_vocab=False)

In [17]:
# ./dataからtrainとtestのtsvファイル（先程作ったもの）を読み込む
train_val_ds, test_ds = torchtext.data.TabularDataset.splits(
    path='./data/', train='IMDb_train.tsv', test='IMDb_test.tsv', format='tsv',
    fields=[('Text', TEXT), ('Label', LABEL)])

In [18]:
# 動作確認
print('訓練及び検証のデータ数', len(train_val_ds))
print('1つ目の訓練および検証のデータ', vars(train_val_ds[0]))

訓練及び検証のデータ数 25000
1つ目の訓練および検証のデータ {'Text': ['i', 'just', 'saw', 'this', 'film', ',', 'i', 'first', 'saw', 'it', 'when', 'i', 'was', '7', 'and', 'could', 'just', 'about', 'remember', 'the', 'end', '.', 'so', 'i', 'watched', 'it', 'like', ',', '10', 'minutes', 'ago', ',', 'and', '(i', 'may', 'seem', 'like', 'a', 'baby', 'as', 'i', 'am', '12', 'ha-ha)', 'i', 'started', 'to', 'cry', 'at', 'the', 'ending', ',', 'i', 'forgotten', 'how', 'sad', 'it', 'was', '.', 'i', 'think', 'i', 'was', 'mainly', 'sad', 'for', 'anne-marie', 'because', 'she', 'said:', "'i", 'love', 'you', "charlie'", 'and', 'also:', "'i'll", 'miss', 'you', "charlie'", ',', 'just', 'made', 'me', 'really', 'cry', 'ha-ha', '.', 'it', 'has', 'to', 'be', 'one', 'of', 'me', 'favourite', 'movies', 'of', 'all', 'time', ',', 'it', 'is', 'just', 'a', 'film', 'well', 'worth', 'watching', '.', 'watch', 'it', 'ha-ha', ',', 'thats', 'all', 'i', 'can', 'say', 'xdbut', ',', 'i', 'love', 'this', 'film', ',', 'its', 'a', 'true', 'classic', '.'

In [19]:
import random
# torchtext.data.Datasetのsplit関数でtrainとvalに分ける
train_ds, val_ds = train_val_ds.split(
    split_ratio=0.8, random_state=random.seed(1234))

# 動作確認
print('訓練データの数', len(train_ds))
print('検証データの数', len(val_ds))
print('１つ目の訓練データ', vars(train_ds[0]))

訓練データの数 20000
検証データの数 5000
１つ目の訓練データ {'Text': ['this', 'movie', 'is', 'pretty', 'awful', 'but', 'i', 'have', 'some', 'interesting', 'information', 'about', 'it:it', 'was', 'filmed', 'in', '1976', 'at', 'northern', 'arizona', 'university', 'in', 'flagstaff', ',', 'az', ',', 'as', 'well', 'as', 'at', 'oak', 'creek', 'canyon', 'near', 'sedona', ',', 'az', '.', 'a', 'good', 'bulk', 'of', 'the', 'extras', 'in', 'the', 'film', 'are', 'then-drama', 'students', 'from', 'nau', '.', 'i', 'was', 'a', 'freshman', 'there', 'that', 'year', ',', 'minoring', 'in', 'theatre', ',', 'but', 'for', 'some', 'reason', 'i', "didn't", 'get', 'involved', 'with', 'the', 'production', '.', 'i', 'did', 'however', 'know', 'several', 'people', 'who', 'did', 'and', 'can', 'supply', 'this', 'rather', 'odd', 'fact:there', 'is', 'a', 'scene', 'in', 'this', 'movie', 'where', 'two', 'of', 'the', 'principals', ',', 'as', 'part', 'of', 'their', 'hazing', 'ritual', ',', 'have', 'to', 'run', 'naked', 'into', 'the', 'woods', '

## ボキャブラリーの作成

In [20]:
# tochtextで単語ベクトルとして英語学習済みモデルを読み込む
from torchtext.vocab import Vectors
english_fasttext_vectors = Vectors(name='./data/wiki-news-300d-1M.vec')

100%|█████████▉| 999987/999994 [02:07<00:00, 7778.56it/s]

In [21]:
# 単語ベクトルの中身
print('１単語を表現する次元数: ', english_fasttext_vectors.dim)
print('単語数: ', len(english_fasttext_vectors))

１単語を表現する次元数:  300
単語数:  999994


In [22]:
# ベクトル化したバージョンのボキャブラリーを作成します
# 
TEXT.build_vocab(train_ds, vectors=english_fasttext_vectors, min_freq=10)

# ボキャブラリーのベクトルを確認します
print(TEXT.vocab.vectors.shape)  # 19169個の単語が300次元のベクトルで表現されている
TEXT.vocab.vectors

# ボキャブラリーの単語の順番を確認します
TEXT.vocab.stoi

torch.Size([19169, 300])


defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7ff1162be780>>,
            {'<unk>': 0,
             '<pad>': 1,
             '<cls>': 2,
             '<eos>': 3,
             'the': 4,
             '.': 5,
             ',': 6,
             'and': 7,
             'a': 8,
             'of': 9,
             'to': 10,
             'is': 11,
             'in': 12,
             'it': 13,
             'i': 14,
             'this': 15,
             'that': 16,
             'was': 17,
             'as': 18,
             'for': 19,
             'with': 20,
             'but': 21,
             'movie': 22,
             'film': 23,
             'on': 24,
             'not': 25,
             'his': 26,
             'you': 27,
             'are': 28,
             'have': 29,
             'he': 30,
             'be': 31,
             'one': 32,
             'at': 33,
             'all': 34,
             'by': 35,
             'an': 36,
             'they': 3

100%|█████████▉| 999987/999994 [02:20<00:00, 7778.56it/s]

In [23]:
# DataLoderの作成
train_dl = torchtext.data.Iterator(train_ds, batch_size=24, train=True)
val_dl = torchtext.data.Iterator(val_ds, batch_size=24, train=False, sort=False)
test_dl = torchtext.data.Iterator(val_ds, batch_size=24, train=False, sort=False)

# 動作確認
batch = next(iter(val_dl))
print(batch.Text)
print(batch.Label)

(tensor([[    2,    78,     6,  ...,     1,     1,     1],
        [    2,    15,    11,  ...,     1,     1,     1],
        [    2,    14,   296,  ...,     1,     1,     1],
        ...,
        [    2,  1038,   409,  ...,     4,  4339,     3],
        [    2,  1319,   931,  ...,  1312,    12,     3],
        [    2,     0, 16934,  ...,    11,    36,     3]]), tensor([117, 148, 186, 207, 186, 121, 204, 123, 207, 143, 256, 160, 135,  69,
        117, 188, 239, 180, 256, 256, 127, 256, 256, 256]))
tensor([1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0])
